## Import relevent packages

In [1]:
import numpy as np
import scipy as sp

from scipy import integrate
from scipy.optimize import basinhopping, fminbound, minimize

from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [163]:
# Functions to create synthetic data
def irt_evaluation(difficulty, discrimination, thetas):
    """
        Evaluates an IRT model and returns the exact values
        
        Args:
            difficulty: [array] of difficulty parameters
            discrimination:  [array | number] of discrimination parameters
            thetas: [array] of person abilities
            
        Returns:
            dichotomous matrix of [difficulty.size x thetas.size] representing
            synthetic data
    """
    # If discrimination is a scalar, make it an array
    if not np.ndim(discrimination):
        discrimination = np.ones_like(difficulty) * discrimination

    kernel = difficulty[:, None] - thetas
    kernel *= discrimination[:, None]
    return 1.0 / (1 + np.exp(kernel))



def create_synthetic_irt_dichotomous(difficulty, discrimination, thetas):
    """
        Creates synthetic IRT data to test parameters estimation
        functions.  Only for use with dichotomous outputs
        
        Args:
            difficulty: [array] of difficulty parameters
            discrimination:  [array | number] of discrimination parameters
            thetas: [array] of person abilities
            
        Returns:
            dichotomous matrix of [difficulty.size x thetas.size] representing
            synthetic data
    """
    continuous_output = irt_evaluation(difficulty, discrimination, thetas)

    # convert to binary based on probability
    random_compare = np.random.rand(*continuous_output.shape)
    
    return random_compare <= continuous_output


def rauch_estimate(dataset, discrimination=1):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            
        Returns:
            array of discrimination estimates
    """
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    return (np.sqrt(1 + np.pi * discrimination**2 / 8) * 
            np.log(n_no / n_yes) / discrimination)


def onepl_estimate_dep(dataset):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    scalar = np.log(n_no / n_yes)

    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        the_sign = (-1)**response
        gauss = 1 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = theta - difficulty[:, None]
        kernel *= discrimination * the_sign[:, None]
        return  gauss * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0)

    # Inline definition of cost function to minimize
    def min_func(estimate):
        difficulty = np.sqrt(1 + np.pi * estimate**2 / 8) * scalar / estimate
        otpt = list(map(lambda ndx: integrate.fixed_quad(quadrature_function, -5, 5, 
                    (difficulty, estimate, unique_sets[:, ndx]), n=181)[0] + 1e-23, 
                        range(unique_sets.shape[1])))
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    discrimination = fminbound(min_func, 0.25, 10)
    
    return discrimination, np.sqrt(1 + np.pi * discrimination**2 / 8) * scalar / discrimination


def onepl_estimate(dataset):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    scalar = np.log(n_no / n_yes)

    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        the_sign = (-1)**response
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = the_sign[:, :, None] * np.ones((1, 1, theta.size))
        kernel *= discrimination   
        kernel *= (theta[None, None, :] - difficulty[:, None, None])
        
        return  gauss[None, :] * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0).squeeze()

    # Inline definition of cost function to minimize
    def min_func(estimate):
        difficulty = np.sqrt(1 + np.pi * estimate**2 / 8) * scalar / estimate
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                                    (difficulty, estimate, unique_sets), n=61)[0]
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    discrimination = fminbound(min_func, 0.25, 10)
    
    return discrimination, np.sqrt(1 + np.pi * discrimination**2 / 8) * scalar / discrimination


def twopl_estimate_dep(dataset):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    scalar = np.log(n_no / n_yes)

    minimizer_kwargs = {'method': 'SLSQP'}
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        the_sign = (-1)**response
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = theta - difficulty[:, None]
        kernel *= discrimination[:, None] * the_sign[:, None]
        return gauss * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0)

    # Inline definition of cost function to minimize
    def min_func(estimate):
        difficulty = np.sqrt(1 + np.pi * estimate**2 / 8) * scalar / estimate
        otpt = list(map(lambda ndx: integrate.fixed_quad(quadrature_function, -5, 5, 
                    (difficulty, estimate, unique_sets[:, ndx]), n=61)[0] + 1e-23, 
                        range(unique_sets.shape[1])))
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    initial_guess = np.ones((dataset.shape[0],))
    discrimination = basinhopping(min_func, initial_guess, niter_success=10,
                                  minimizer_kwargs=minimizer_kwargs)['x']
    
    return discrimination, np.sqrt(1 + np.pi * discrimination**2 / 8) * scalar / discrimination


def twopl_estimate(dataset, max_iter=25):
    """
        Estimates the difficulty parameters via the approximation
    
        Args:
            dataset: [items x participants] matrix of True/False Values
            max_iter:  maximum number of iterations to run
            
        Returns:
            array of discrimination, difficulty estimates
    """
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    scalar = np.log(n_no / n_yes)

    n_items = dataset.shape[0]
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        the_sign = (-1)**response
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = the_sign[:, :, None] * np.ones((1, 1, theta.size))
        kernel *= discrimination[:, None, None]       
        kernel *= (theta[None, None, :] - difficulty[:, None, None])
        
        return  gauss[None, :] * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0).squeeze()

    # Inline definition of cost function to minimize
    def min_func(estimate):
        difficulty = np.sqrt(1 + np.pi * estimate**2 / 8) * scalar / estimate
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                                    (difficulty, estimate, unique_sets), n=61)[0]
        return -np.log(otpt).dot(counts)
       
    # Perform the minimization
    initial_guess = np.ones((dataset.shape[0],))
    previous_guess = initial_guess.copy()
    
    for iteration in range(max_iter):
        for ndx in range(n_items):
            def min_func_local(estimate):
                initial_guess[ndx] = estimate
                return min_func(initial_guess)

            initial_guess[ndx] = fminbound(min_func_local, 0.25, 6, xtol=1e-3)
        
        previous_guess = initial_guess.copy()
        if np.abs(initial_guess - previous_guess).max() < 1e-3:
            break
        
        
    
    return initial_guess, np.sqrt(1 + np.pi * initial_guess**2 / 8) * scalar / initial_guess


def rauch_estimate_full_dep(dataset, discrimination=1):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            
        Returns:
            array of discrimination estimates
    """
    minimizer_kwargs = {'method': 'SLSQP'}
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, response):
        the_sign = (-1)**response
        gauss = 1 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = theta - difficulty[:, None]
        kernel *= discrimination * the_sign[:, None]
        return gauss * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0)

    # Inline definition of cost function to minimize
    def min_func(difficulty):
        otpt = list(map(lambda ndx: integrate.fixed_quad(quadrature_function, -5, 5, 
                    (difficulty, unique_sets[:, ndx]), n=61)[0] + 1e-23, range(unique_sets.shape[1])))
        return -np.log(otpt).dot(counts)
    
    # Perform the minimization
    initial_guess = np.zeros((dataset.shape[0],))
    return basinhopping(min_func, initial_guess, niter_success=10,
                        minimizer_kwargs=minimizer_kwargs)['x']


def rauch_estimate_full(dataset, discrimination=1, max_iter=25):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            discrimination: scalar of discrimination used in model (default to 1)
            max_iter: maximum number of iterations to run
            
        Returns:
            array of discrimination estimates
    """
    n_items = dataset.shape[0]
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, response):
        the_sign = (-1)**response
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = the_sign[:, :, None] * np.ones((1, 1, theta.size))
        kernel *= discrimination
        kernel *= (theta[None, None, :] - difficulty[:, None, None])
        
        return gauss[None, :] * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0).squeeze()

    # Inline definition of cost function to minimize
    def min_func(difficulty):
        otpt = integrate.fixed_quad(quadrature_function, -5, 5, 
                (difficulty, unique_sets), n=61)[0] + 1e-23
        return -np.log(otpt).dot(counts)

    initial_guess = rauch_estimate(dataset, discrimination=discrimination)

    for iteration in range(max_iter):
        previous_guess = initial_guess.copy()
        
        for ndx in range(n_items):
            # Minimize each one separately
            value = initial_guess[ndx] * 1.0
            def min_func_local(estimate):
                initial_guess[ndx] = estimate
                return min_func(initial_guess)
            
            initial_guess[ndx] = fminbound(min_func_local, 
                                           value-0.75,
                                           value+0.75)

        print(iteration, np.abs(initial_guess - previous_guess).max())
        if(np.abs(initial_guess - previous_guess).max() < 0.001):
            break
            
    # Perform the minimization
    return initial_guess


def onepl_estimate_full(dataset):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    minimizer_kwargs = {'method': 'SLSQP'}
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        the_sign = (-1)**response
        gauss = 1 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = theta - difficulty[:, None]
        kernel *= discrimination * the_sign[:, None]
        return gauss * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0)

    # Inline definition of cost function to minimize
    def min_func(estimate):
        otpt = list(map(lambda ndx: integrate.fixed_quad(quadrature_function, -5, 5, 
                    (estimate[1:], estimate[0], unique_sets[:, ndx]), n=61)[0] + 1e-23, 
                        range(unique_sets.shape[1])))
        return -np.log(otpt).dot(counts)
    
    # Perform the minimization
    initial_guess = np.zeros((dataset.shape[0] + 1,))
    initial_guess[0] = 1.0
    results =  basinhopping(min_func, initial_guess, niter_success=10,
                            minimizer_kwargs=minimizer_kwargs)['x']
    
    return results[0], results[1:]


def twopl_estimate_full(dataset):
    """
        Estimates parameters in an IRT model with full        
        gaussian quadrature
        
        Args:
            dataset: [items x participants] matrix of True/False Values
            
        Returns:
            array of discrimination, difficulty estimates
    """
    minimizer_kwargs = {'method': 'SLSQP'}
    unique_sets, counts = np.unique(dataset, axis=1, return_counts=True)
    n_parameters = dataset.shape[0]

    # Inline definition of quadrature function
    def quadrature_function(theta, difficulty, discrimination, response):
        the_sign = (-1)**response
        gauss = 1.0 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
        kernel = theta - difficulty[:, None]
        kernel *= discrimination[:, None] * the_sign[:, None]
        return gauss * (1.0 / (1.0 + np.exp(kernel))).prod(axis=0)

    # Inline definition of cost function to minimize
    def min_func(estimate):
        otpt = list(map(lambda ndx: integrate.fixed_quad(quadrature_function, -5, 5, 
                    (estimate[n_parameters:], estimate[:n_parameters], 
                     unique_sets[:, ndx]), n=61)[0] + 1e-23, 
                        range(unique_sets.shape[1])))
        return -np.log(otpt).dot(counts)
    
    # Perform the minimization
    initial_guess = np.zeros((2 * dataset.shape[0],))
    initial_guess[:n_parameters] = 1.0
    results =  basinhopping(min_func, initial_guess, niter_success=10,
                            minimizer_kwargs=minimizer_kwargs)['x']
    
    return results[:n_parameters], results[n_parameters:]


def discriminate_quadrature(discriminate, theta, mu, scalar, dataset):
    the_sign = (-1)**dataset
    gauss = 1.0 / np.sqrt(2*np.pi * .25) * np.exp(-np.square(discriminate - mu) / 2 / .25)
    otpt = 1.0 / (1 + np.exp(theta * np.outer(the_sign, discriminate) - 
                             np.outer(scalar * the_sign, np.sqrt(1 + 1/8 * np.pi * discriminate**2))))
    otpt *= gauss[None, :]
    return otpt


def discriminate_quadrature2(discriminate, theta, mu, scalar, dataset):
    the_sign = (-1)**dataset
    gauss = 1.0 / np.sqrt(2*np.pi * .25) * np.exp(-np.square(mu[:, None] - discriminate ) / 2 / .25)
    otpt = 1.0 / (1 + np.exp(theta * np.outer(the_sign, discriminate) - 
                             np.outer(scalar * the_sign, np.sqrt(1 + 1/8 * np.pi * discriminate**2))))
    otpt *= gauss
    return otpt

def discriminate_quadrature3(discriminate, theta, mu, scalar, dataset):
    the_sign = (-1)**dataset
    gauss = 1.0 / np.sqrt(2*np.pi * .25) * np.exp(-np.square(mu[:, None] - discriminate ) / 2 / .25)
    
    g = scalar / np.sqrt(1 + np.pi * mu**2 / 8) * the_sign
    
    otpt = 1.0 / (1 + np.exp(np.outer(theta * the_sign - mu * np.pi * g / 8, discriminate) - g[:, None]))
#     otpt = 1.0 / (1 + np.exp(theta * np.outer(the_sign, discriminate) - 
#                              np.outer(scalar * the_sign, np.sqrt(1 + 1/8 * np.pi * discriminate**2))))
    otpt *= gauss
    return otpt


def estimate_theta(dataset, discrimination):
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)
    scalar = np.log(n_no / n_yes)
    
    # Perform the integral
    
    

    
    
# def onepl_estimate_nob(dataset):
#     """May add if works"""
    
#     def min_func(estimate):
        
    
#     a_estimate = fminbound(min_func, 0.25, 10)
    
#     return a_estimate, rauch_estimate(dataset, a_estimate)
    

In [173]:
n_items, n_participants = 20, 20000
diffc = np.linspace(-2.3, 2.3, n_items)#np.random.rand(n_items,) * 4 - 2
discr = 1.7#0.5 + 2.0 * np.random.rand(n_items,)
thetas = np.random.randn(n_participants)

In [146]:
20000 * 100 * 61 * 8 / 1024 / 1024

930.7861328125

In [83]:
v = np.array([1, 2, 3, 4]).reshape(2, 2)[:, :, None] * np.ones((1, 1, 3))

In [86]:
g = v * np.array([2, 2])[:, None, None]

In [91]:
g[:, :, 1]

array([[2., 4.],
       [6., 8.]])

In [174]:
syn_data = create_synthetic_irt_dichotomous(diffc, discr, thetas)

In [32]:
from time import time

In [179]:
t1 = time()
a, b = onepl_estimate(syn_data)
print(time() - t1)
a, b

0.6937539577484131


(1.6788175172859998,
 array([-2.43249306, -2.14808459, -1.88300469, -1.59413961, -1.33196776,
        -1.08398148, -0.833581  , -0.58718675, -0.36300546, -0.11238106,
         0.10838774,  0.33509576,  0.58873628,  0.82818074,  1.08623536,
         1.34182109,  1.59670812,  1.8698412 ,  2.17647234,  2.44471322]))

In [176]:
t1 = time()
a = rauch_estimate_full(syn_data, discrimination=1.7)
print(time() - t1)
a

0 0.11266085652373103
1 0.008931068791670826
2 0.002263408781289611
3 0.0011455182566005445
4 0.0008187290205051712
52.64837193489075


array([-2.29852276, -2.05839962, -1.82760255, -1.56794423, -1.3247466 ,
       -1.08823577, -0.84347176, -0.59756474, -0.3704165 , -0.11398726,
        0.11271552,  0.34494102,  0.60244756,  0.8418718 ,  1.09464119,
        1.33870795,  1.57538654,  1.82140753,  2.08847884,  2.31466455])

In [177]:
diffc

array([-2.3       , -2.05789474, -1.81578947, -1.57368421, -1.33157895,
       -1.08947368, -0.84736842, -0.60526316, -0.36315789, -0.12105263,
        0.12105263,  0.36315789,  0.60526316,  0.84736842,  1.08947368,
        1.33157895,  1.57368421,  1.81578947,  2.05789474,  2.3       ])

In [106]:
t1 = time()
a, b = twopl_estimate(syn_data)
print(time() - t1)
a,b

276.0169632434845


(array([1.13312871, 1.21091616, 0.58124213, 1.61126301, 0.87666526,
        1.83183851, 1.16075654, 2.26795425, 1.74207986, 0.65951824,
        0.57785906, 2.34438872, 2.4051655 , 1.30564873, 1.34984211]),
 array([-2.13619508, -1.84781525, -1.59963582, -1.20896183, -0.82286719,
        -0.67861032, -0.36356409,  0.07515113,  0.32690654,  0.85210223,
         1.07495036,  1.43102268,  1.69078015,  2.00492757,  2.34379097]))

In [121]:
fminbound?

In [27]:
rauch_estimate(syn_data, 3.76)

array([-2.63733723, -2.20825111, -1.78479727, -1.37855605, -1.02720543,
       -0.67796172, -0.33398669, -0.02287565,  0.31914057,  0.64988846,
        1.0024752 ,  1.36869738,  1.76826541,  2.21392393,  2.65112939])

In [7]:
n_no = np.count_nonzero(~syn_data, axis=1)
n_yes = np.count_nonzero(syn_data, axis=1)
scalar = np.log(n_no / n_yes)

def min_outside(discriminate):
    theta_estimate = list(map(lambda ndx: fminbound(min_func, -6, 6, 
                                                    args=(discriminate, scalar, syn_data[:, ndx])), 
                              range(syn_data.shape[1])))
    diffclty_est = rauch_estimate(syn_data, discriminate)
    otpt = irt_evaluation(diffclty_est, discriminate, np.array(theta_estimate))
    otpt = np.where(syn_data, otpt, 1.0 - otpt)
    return -1 * np.log(otpt + 1e-23).sum()
    

def min_func(theta, discrimination, scalar, dataset):
    otpt = integrate.fixed_quad(discriminate_quadrature, .1, 6, 
                                (theta, discrimination, scalar, dataset), n=181)[0]
                              
    return -1 * np.log(otpt + 1e-23).sum()


def min_outside2(discriminate):
    theta_estimate = list(map(lambda ndx: fminbound(min_func2, -6, 6, 
                                                    args=(discriminate, scalar, syn_data[:, ndx])), 
                              range(syn_data.shape[1])))
    diffclty_est = rauch_estimate(syn_data, discriminate)
    otpt = irt_evaluation(diffclty_est, discriminate, np.array(theta_estimate))
    otpt = np.where(syn_data, otpt, 1.0 - otpt)
    return -1 * np.log(otpt + 1e-23).sum()

def min_func2(theta, discrimination, scalar, dataset):
    otpt = integrate.fixed_quad(discriminate_quadrature2, .1, 6, 
                                (theta, discrimination, scalar, dataset), n=181)[0]
                              
    return -1 * np.log(otpt + 1e-23).sum()


def min_outside3(discriminate):
    theta_estimate = list(map(lambda ndx: fminbound(min_func3, -6, 6, 
                                                    args=(discriminate, scalar, syn_data[:, ndx])), 
                              range(syn_data.shape[1])))
    diffclty_est = rauch_estimate(syn_data, discriminate)
    otpt = irt_evaluation(diffclty_est, discriminate, np.array(theta_estimate))
    otpt = np.where(syn_data, otpt, 1.0 - otpt)
    return -1 * np.log(otpt + 1e-23).sum()

def min_func3(theta, discrimination, scalar, dataset):
    otpt = integrate.fixed_quad(discriminate_quadrature3, .1, 6, 
                                (theta, discrimination, scalar, dataset), n=181)[0]
                              
    return -1 * np.log(otpt + 1e-23).sum()

In [1268]:
integrate.fixed_quad(discriminate_quadrature2, .1, 6, 
                                (thetas[0], results['x'], scalar, syn_data[:, 0]), n=181)[0]

ValueError: operands could not be broadcast together with shapes (100,181) (75,181) (100,181) 

In [1257]:
integrate.fixed_quad(discriminate_quadrature3, .1, 6, 
                                (thetas[1], results['x'], scalar, syn_data[:, 1]), n=181)[0]

array([0.85511211, 0.47321636, 0.99920989, 0.55370772, 0.99223381,
       0.80221421, 0.97640611, 0.41839576, 0.61560956, 0.6781604 ,
       0.82426715, 0.88349642, 0.35177828, 0.96550339, 0.23648727,
       0.86032907, 0.89304573, 0.99231887, 0.60644315, 0.94562986,
       0.61973838, 0.4063197 , 0.93088941, 0.75633204, 0.67466477,
       0.46606003, 0.6792932 , 0.98473134, 0.28016737, 0.89534628,
       0.3904023 , 0.5684692 , 0.84332523, 0.90358576, 0.2557344 ,
       0.81304824, 0.82446366, 0.67013363, 0.84557239, 0.68854207,
       0.88721827, 0.76702934, 0.88002096, 0.98441433, 0.66573812,
       0.87678077, 0.74136814, 0.95127031, 0.97692143, 0.86921238,
       0.80537627, 0.22299954, 0.60809085, 0.93738552, 0.8268099 ,
       0.9030025 , 0.68438108, 0.50438262, 0.94453346, 0.92033268,
       0.56637183, 0.84912703, 0.86223347, 0.77442497, 0.8439433 ,
       0.92131256, 0.52406889, 0.56828119, 0.3785709 , 0.52838508,
       0.04912343, 0.77773897, 0.56369984, 0.91583809, 0.38630

In [1258]:
theta = thetas[1]
sigma_ao = 0.25
the_sign = (-1)**syn_data[:, 1]
g = scalar / np.sqrt(1 + results['x']**2 * np.pi / 8)
a_tilde = np.pi * results['x'] / 8 * g - theta
b_tilde = 8 * g / (8 * theta - np.pi * results['x'] * g) - results['x']

1.0 / (1 + np.exp(the_sign * b_tilde * a_tilde / np.sqrt(1 + np.pi / 8 * sigma_ao * a_tilde**2)))

array([0.85814936, 0.47327575, 0.99875062, 0.65031295, 0.99045188,
       0.8033832 , 0.97453525, 0.44816864, 0.61575087, 0.67820574,
       0.83046809, 0.88341017, 0.37165111, 0.964237  , 0.3204893 ,
       0.8603453 , 0.89287018, 0.99032523, 0.60992243, 0.94486577,
       0.7069605 , 0.60466161, 0.93074577, 0.75746388, 0.67470202,
       0.59839719, 0.67974085, 0.98270715, 0.28278064, 0.8959302 ,
       0.39088174, 0.56881611, 0.85208704, 0.90398657, 0.25603308,
       0.83912438, 0.8604158 , 0.67228594, 0.89989725, 0.87641127,
       0.88709412, 0.76974822, 0.88289121, 0.98238233, 0.66582051,
       0.87783006, 0.75216953, 0.94989537, 0.9749738 , 0.89980333,
       0.82064985, 0.35527382, 0.6081747 , 0.93667179, 0.83620478,
       0.90274939, 0.68655319, 0.50443307, 0.94334616, 0.92202585,
       0.68869459, 0.84921267, 0.8622428 , 0.83347795, 0.87013508,
       0.93331106, 0.52798669, 0.5688353 , 0.39948572, 0.54039763,
       0.05039422, 0.77854632, 0.56462665, 0.91544176, 0.46067

In [1259]:
results['x']

array([1.41492941, 1.93553303, 3.99996771, 0.62386648, 2.57952744,
       1.59571121, 2.34264808, 0.87752847, 1.85105384, 2.36111658,
       1.28923622, 3.06633157, 0.94693447, 2.40820066, 0.46759531,
       3.12153972, 2.36511445, 2.24906095, 1.36196545, 2.81475972,
       0.65217807, 0.30001588, 1.70306548, 1.58110817, 3.1054116 ,
       0.45983567, 1.70602631, 2.30295467, 1.32584821, 1.60150085,
       1.62681333, 1.71704759, 1.22721333, 1.61223204, 1.58921701,
       1.00366844, 0.9373985 , 1.45454305, 0.86264523, 0.47855657,
       2.15706441, 1.43545344, 1.41643308, 2.28669946, 1.98243029,
       1.56512177, 1.16584862, 1.99747487, 2.23242984, 0.99645689,
       1.11241299, 0.30212327, 2.02598522, 2.40859883, 1.22419943,
       2.06659262, 1.44971293, 1.9683699 , 2.14008874, 1.45721464,
       0.56293747, 2.09004653, 3.29845925, 0.80825143, 1.02370456,
       1.17437576, 1.30048133, 1.64932416, 0.94620598, 1.11360106,
       2.33816454, 1.6736481 , 1.55370824, 2.0356017 , 0.61978

In [1269]:
dsc = fminbound(min_outside, .1, 10)
print(dsc)

1.4037308483158024


In [1270]:
# minimizer_kwargs = {'method': 'SLSQP'}
# results =  basinhopping(min_outside2, np.ones((syn_data.shape[0],)), niter_success=10,
#                         minimizer_kwargs=minimizer_kwargs)['x']
results = minimize(min_outside2, 1.43 * np.ones((syn_data.shape[0],)), method="SLSQP")

In [1271]:
results['x']

array([2.0300451 , 1.85352898, 0.96708373, 1.66136938, 1.94167675,
       1.25060844, 1.28230609, 2.59848612, 1.03609674, 2.46543389,
       0.79486489, 1.76906577, 0.68583572, 1.24402741, 2.56512222,
       1.5865441 , 1.80768186, 2.13974918, 0.84836954, 0.72607153,
       1.03941558, 2.2320473 , 1.39568767, 2.63621078, 1.7262962 ,
       1.37150454, 1.10671649, 2.09426724, 1.49181596, 2.26111207,
       0.80666561, 1.79271931, 2.42863214, 1.35280144, 2.06250453,
       1.15258158, 0.91728467, 0.99162475, 2.4662353 , 0.76836283,
       1.89628112, 0.90859178, 1.47077553, 2.50010384, 0.6295446 ,
       1.77277428, 0.43487684, 0.9714011 , 1.27145363, 1.31562231,
       2.33814699, 2.21404856, 1.10131796, 0.69876167, 2.25583521,
       0.66690814, 1.99194796, 2.32804012, 1.42252323, 2.00458785,
       2.66140625, 0.76905158, 2.08006693, 2.02861151, 0.52095526,
       0.85238718, 0.84721971, 0.798873  , 2.25007009, 2.2604056 ,
       0.71677002, 1.74114289, 0.87491748, 1.53650895, 1.41958

In [1272]:
bounds = [(.3, 4),] * syn_data.shape[0]
results = minimize(min_outside3, 1.52 * np.ones((syn_data.shape[0],)), method="SLSQP", bounds=bounds)

In [1273]:
(results['x'] - discr)

array([ 0.10349609, -0.17556523, -0.0812461 , -0.1419163 , -0.41840906,
       -0.30970809, -0.03411393,  0.12215547,  0.07962876,  0.54841904,
        0.02765276,  0.12152264, -0.25212861, -0.27537895,  0.09410494,
       -0.05256524,  0.16519631,  0.36883865, -0.00234148, -0.088737  ,
        0.02567914,  0.54770262, -0.09844973,  0.18473026, -0.15154735,
       -0.28012269, -0.15630157, -0.13633508, -0.16591904, -0.20713712,
       -0.22156739, -0.2925401 , -0.05851916,  0.03386905,  0.44380911,
       -0.175499  ,  0.01700538,  0.02142917,  0.27378773,  0.04905027,
        0.35678001,  0.08886857,  0.26123894,  0.13048201, -0.10873259,
       -0.00650124, -0.1090013 , -0.07704953,  0.15150939, -0.16952299,
        0.3309674 ,  0.36216659, -0.30332208,  0.00580663,  0.12210379,
       -0.0946305 , -0.22505513, -0.04522457,  0.09858839, -0.07737156,
        0.30032025, -0.03387459,  0.56404986,  0.08800781, -0.0678549 ,
       -0.12190695,  0.04713425, -0.02132307,  0.22916489, -0.02

In [1274]:
discr

array([1.92691508, 2.02949809, 1.05092081, 1.80259153, 2.35535918,
       1.56113761, 1.31770492, 2.47650231, 0.95730568, 1.91542605,
       0.76878573, 1.64564705, 0.93940572, 1.52035384, 2.46500476,
       1.63635693, 1.64395199, 1.76743588, 0.8519046 , 0.81594435,
       1.01559248, 1.6853837 , 1.4940899 , 2.44210016, 1.87944959,
       1.6479541 , 1.26469403, 2.22531097, 1.65585319, 2.46375817,
       1.02995968, 2.08467957, 2.4809909 , 1.32047885, 1.61971788,
       1.33015118, 0.90206115, 0.97222251, 2.18419284, 0.72134291,
       1.53866583, 0.82132323, 1.21053255, 2.35535905, 0.73994676,
       1.77612031, 0.54377067, 1.05002376, 1.12176457, 1.48630331,
       2.00575801, 1.85193298, 1.40616633, 0.69434602, 2.1316881 ,
       0.76341747, 2.21288101, 2.37395947, 1.32461171, 2.08240231,
       2.36023791, 0.80466648, 1.5136733 , 1.93823826, 0.59005409,
       0.97550823, 0.80131346, 0.82274824, 2.01842209, 2.28065186,
       0.93607144, 1.71508885, 0.75980908, 2.08227082, 1.59804

In [1253]:
minimize?

In [1027]:
theta_estimate = list(map(lambda ndx: fminbound(min_func, -6, 6, 
                                                args=(discr, scalar, syn_data[:, ndx])), 
                          range(syn_data.shape[1])))


In [1028]:
np.mean(thetas-theta_estimate), np.std(thetas-theta_estimate, ddof=1)

(0.11107225015581378, 0.6326116860920391)

In [1029]:
min_outside(dsc), min_outside(discr)

(5913.806906521615, 5918.406409628524)

In [1169]:
onepl_estimate(syn_data)

(1.514085548957218,
 array([ 1.20627358, -1.08985202, -0.28639781, -0.50806979, -0.06558048,
         1.59362194,  1.08985202,  1.7633512 , -0.74555259, -1.59362194,
         1.66793925,  1.73072106, -0.70306101, -0.62834338, -1.48290067,
        -1.48290067, -0.08747007,  1.38055312, -0.56360202, -1.48290067,
         0.59579084,  1.77999967, -0.61201997, -0.05099833,  0.28639781,
         1.90350771, -0.87802589,  0.16065413,  1.45659448, -0.33890408,
         0.01456745, -1.2508085 , -0.76276479,  0.16799719,  1.27357028,
        -1.56505747,  1.74692737,  0.10207679,  0.80637354, -0.76276479,
         0.73699377, -1.33203624, -1.14174453, -1.44362864, -0.28639781,
         0.99054191, -0.1753457 , -0.87802589,  0.78882797, -0.31633808,
        -0.01456745, -0.62016968, -1.19533584, -0.78010682,  0.12401209,
        -0.57161656, -0.58771057,  0.57965261, -1.38055312,  0.4767519 ,
         0.85086906, -1.2508085 , -0.84189519, -0.4072538 , -0.99054191,
         1.45659448, -1.5097277

In [1091]:
discr.mean()

1.9595483704090835

In [1166]:
onepl_estimate_full(syn_data)

KeyboardInterrupt: 

In [892]:
20 * 200 / 17 / 188

1.2515644555694618

In [533]:
onepl_estimate_full(syn_data)

(3.883078090154843,
 array([-0.40863746,  1.70804849,  1.66386809, -1.17871092, -0.78549423,
         0.89697624, -0.6491721 ,  0.322759  , -0.06428948, -1.04881811]))

In [534]:
diffc

array([-0.53700691,  1.86937898,  1.92288603, -1.28300466, -0.83887183,
        0.99258767, -0.72174919,  0.31027538, -0.08999906, -1.09527454])

In [391]:
no = np.count_nonzero(syn_data, axis=0)

In [219]:
rauch_estimate(syn_data, 1.63)

array([-1.42837668,  0.48182139,  1.57388101, -0.96813419,  0.51029216])

In [260]:
onepl_estimate(syn_data)

(1.4822199280344053,
 array([-0.11062872, -0.16619346,  0.4312827 ,  0.67300025, -0.45077536,
         0.18477731,  0.03683686,  0.03683686,  0.4118845 , -2.02320149,
        -0.2220617 ,  0.84694451,  0.39257579, -0.2220617 ,  1.08680977,
        -0.49006454,  0.84694451, -0.47036756,  0.24077016,  0.        ,
        -0.16619346,  0.4312827 , -0.2220617 ,  1.13880939, -0.18477731,
        -0.869659  , -0.4312827 , -0.67300025,  0.61076044, -1.11261136,
        -0.45077536,  0.12912299, -1.11261136, -0.39257579,  1.01160074,
         0.25952817, -1.33516745, -0.18477731,  0.45077536,  0.73674879,
        -1.01160074, -0.93934418,  0.03683686, -0.2220617 , -0.16619346,
        -0.91584715,  0.4118845 , -1.46004349, -0.0552645 ,  0.09215672,
        -0.18477731, -0.11062872, -0.63135161,  1.11261136, -0.33513884,
        -0.49006454,  0.869659  , -0.31614087,  0.67300025, -0.09215672,
        -0.16619346, -0.16619346, -0.09215672, -1.39625077, -0.50987168,
         0.24077016,  0.129122

In [441]:
onepl_estimate_full(syn_data)

(5.335418934560034, array([-1.16398914, -1.23998338]))

In [438]:
diffc

array([-1.15275048, -1.09630471])

In [203]:
rauch_estimate_full(syn_data, 1.63)

array([-0.87030798,  0.15986993,  1.85142745, -1.35159617, -1.35159559])

In [1140]:
twopl_estimate(syn_data)

(array([1.33291769, 1.3770095 , 0.98064735, 2.20780908, 1.28685189,
        1.7695484 , 1.74803377, 1.19393283, 1.30906159, 1.07575171,
        1.48081366, 0.38266531, 1.69659085, 2.51899771, 1.48836964,
        2.74474013, 1.75478913, 1.40927636, 1.01108824, 1.43850133,
        1.15531041, 1.40946524, 1.8571191 , 1.66600621, 1.69465953]),
 array([-1.85822968,  1.66384511, -1.10089609,  0.51286484, -0.75247732,
        -0.59759033, -0.23915277, -1.04042434,  0.05930106, -0.71871478,
        -1.71121401, -1.08962085, -0.49498435,  0.37893994, -0.60940699,
         1.77038621,  0.79998173,  1.56981622,  2.57261698, -1.66067695,
         2.40868801,  1.01488193,  0.35225488, -0.73524004, -0.51398653]))

In [1143]:
results['x']

array([1.26755902, 1.40712429, 0.92518359, 2.62006752, 1.33401342,
       1.8826979 , 1.87175319, 1.26056931, 1.44358028, 1.10899399,
       1.43571467, 0.34170979, 1.77872366, 6.68447951, 1.56634072,
       3.00883417, 2.11993845, 1.37433813, 0.8273699 , 1.39520183,
       1.01118328, 1.63407343, 2.22805181, 1.67096611, 1.76559164])

In [1141]:
discr

array([1.56702704, 1.66692852, 1.04398708, 1.95377561, 1.19232132,
       1.93740192, 1.4438194 , 1.21886136, 1.31546162, 1.13670171,
       1.54962316, 0.50838183, 1.53567551, 1.90708299, 1.55558351,
       3.29702897, 2.13594027, 1.7536264 , 1.47137658, 1.38485356,
       1.63078686, 1.41209839, 1.75629118, 1.41293637, 1.82469343])

In [1144]:
twopl_estimate_full(syn_data)

(array([1.3354512 , 1.41123588, 0.98962051, 2.25528199, 1.2948158 ,
        1.77893023, 1.76915075, 1.20035129, 1.32570921, 1.08696624,
        1.4838015 , 0.38860297, 1.7052068 , 2.55619973, 1.49669549,
        3.01849377, 1.78062537, 1.44520362, 1.03332673, 1.44106226,
        1.19718967, 1.44140106, 1.88460799, 1.67503116, 1.70207364]),
 array([-1.84704293,  1.62681919, -1.10684629,  0.52650766, -0.75623705,
        -0.5928771 , -0.22660388, -1.04794394,  0.07158255, -0.72131831,
        -1.70014523, -1.07954882, -0.48950818,  0.39640319, -0.60886078,
         1.60193057,  0.80971277,  1.53810092,  2.49563483, -1.65420683,
         2.30440648,  1.01778332,  0.3705126 , -0.73416159, -0.50910946]))

In [1171]:
full = lambda a, theta, ti: 1.0 / (1.0 + np.exp(-a * theta + np.sqrt(1 + np.pi * a**2 / 8) * ti))
def approx(a, theta, ti, ao):
    z = 


In [264]:
syn_data

array([[False,  True, False, ..., False,  True, False],
       [False, False, False, ...,  True, False,  True],
       [ True,  True, False, ..., False, False,  True],
       ...,
       [ True, False,  True, ...,  True, False, False],
       [ True,  True,  True, ..., False, False, False],
       [False, False,  True, ..., False, False, False]])

In [308]:
def test_this(dataset, estimate):
    n_no = np.count_nonzero(~dataset, axis=1)
    n_yes = np.count_nonzero(dataset, axis=1)

    difficulty = rauch_estimate(dataset, estimate)
    squared_one = 1.0 / (1.0 + np.exp(difficulty * estimate / 
                                      np.sqrt(1 + np.pi * estimate**2 / 8)))
    squared_zero = 1.0 - squared_one

    return ((n_yes * np.square(squared_zero) + n_no * np.square(squared_one)).sum())

In [309]:
for est in np.linspace(0.1, 2.1, 21):
    print(est.round(3), test_this(syn_data, est))

0.1 3396.605
0.2 3396.605
0.3 3396.605
0.4 3396.605
0.5 3396.605
0.6 3396.605
0.7 3396.605
0.8 3396.605
0.9 3396.605
1.0 3396.605
1.1 3396.605
1.2 3396.605
1.3 3396.605
1.4 3396.605
1.5 3396.605
1.6 3396.605
1.7 3396.605
1.8 3396.605
1.9 3396.605
2.0 3396.605
2.1 3396.605


/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log


In [312]:
onepl_estimate(syn_data)

(3.512490188389685, array([-0.25051613, -1.24945791, -0.19399719]))

In [313]:
discr

3.76

In [314]:
diffc

array([-0.27548552, -1.28618144, -0.26580374])

In [752]:
import numpy as np
from scipy import stats
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [16]:
import json
import numpy as np
import scipy.stats as sp


def one_sample_ttest(group1, mean):
    """Computes a ttest hypothesis for one sample.

    Args:
        group1:  Sample group
        mean: the hypothetical mean of the population

    Returns:
        structure of ttest values
    """
    mean1 = group1.mean()
    std1 = group1.std(ddof=1)
    deg_of_freedom = group1.size - 1

    cohen_d = (mean1 - mean) / std1
    t_metric = cohen_d * np.sqrt(group1.size)

    #Two-tailed
    p_value = sp.t.sf(np.abs(t_metric), deg_of_freedom) * 2
    ci_scalar = sp.t.isf(0.025, deg_of_freedom) * std1 / np.sqrt(group1.size)

    output = {'Mean1': mean1, 'Std1': std1, 'n1': group1.size,
              'Mean2': mean, 'Std2': 0, 'n2': 1,
              'T_value': t_metric, 'P_value': p_value, 'scalar': ci_scalar,
              'Cohen_d': (mean1 - mean) / std1,
              'df': deg_of_freedom}

    return output


def basic_ttest(group1, group2):
    """Computes the t-test for two groups

    assumes equal_variance between the groups

    Args:
        group1:  First group for comparison
        group2:  Second group for comparison

    Returns:
        structure of ttest values

    """
    mean1 = group1.mean()
    mean2 = group2.mean()

    deg_of_freedom = group1.size + group2.size - 2
    pooled_var = group1.var() * group1.size + group2.var() * group2.size
    pooled_var /= deg_of_freedom

    standard_error = np.sqrt(pooled_var * (1. / group1.size + 1. / group2.size))
    t_metric = (mean1 - mean2) / standard_error

    # Two-tailed
    p_value = sp.t.sf(np.abs(t_metric), deg_of_freedom) * 2
    ci_scalar = sp.t.isf(0.025, deg_of_freedom) * standard_error
    output = {'Mean1': mean1, 'Std1': group1.std(ddof=1), 'n1': group1.size,
              'Mean2': mean2, 'Std2': group2.std(ddof=1), 'n2': group2.size,
              'T_value': t_metric, 'P_value': p_value, 'scalar': ci_scalar,
              'Cohen_d': (mean1 - mean2) / np.sqrt(pooled_var),
              'df': deg_of_freedom}

    return output


def repeated_ttest(group1, group2):
    """Computes the t-test for two groups

    assumes equal_variance between the groups and repeated measures

    Args:
        group1:  First group for comparison
        group2:  Second group for comparison

    Returns:
        structure of ttest values
    """
    if group1.size != group2.size:
        raise AssertionError("Repteated TTest groups must be the same size.")

    pairs = group1 - group2
    mean = pairs.mean()
    std = pairs.std(ddof=1)
    deg_of_freedom = group1.size - 1

    standard_error = std / np.sqrt(group1.size)
    t_metric = mean / standard_error

    # Two-tailed
    p_value = sp.t.sf(np.abs(t_metric), deg_of_freedom) * 2
    ci_scalar = sp.t.isf(0.025, deg_of_freedom) * standard_error
    output = {'Mean1': group1.mean(), 'Std1': group1.std(ddof=1), 'n1': group1.size,
              'Mean2': group2.mean(), 'Std2': group2.std(ddof=1), 'n2': group2.size,
              'T_value': t_metric, 'P_value': p_value, 'scalar': ci_scalar,
              'Cohen_d': mean / std, 'df': deg_of_freedom}

    return output


def welch_ttest(group1, group2):
    """Computes the t-test for two groups

    assumes nothing about variance between the groups

    Args:
        group1:  First group for comparison
        group2:  Second group for comparison

    Returns:
        structure of ttest values

    """
    mean1 = group1.mean()
    mean2 = group2.mean()

    variance1 = group1.var(ddof=1) / group1.size
    variance2 = group2.var(ddof=1) / group2.size
    weighted_variance = variance1 + variance2

    deg_of_freedom = np.square(weighted_variance)
    deg_of_freedom /= (np.square(variance1) / (group1.size - 1) +
                       np.square(variance2) / (group2.size - 1))

    t_metric = (mean1 - mean2) / np.sqrt(weighted_variance)

    # Two-tailed
    p_value = sp.t.sf(np.abs(t_metric), deg_of_freedom) * 2
    ci_scalar = sp.t.isf(0.025, deg_of_freedom) * np.sqrt(weighted_variance)
    output = {'Mean1': mean1, 'Std1': group1.std(ddof=1), 'n1': group1.size,
              'Mean2': mean2, 'Std2': group2.std(ddof=1), 'n2': group2.size,
              'T_value': t_metric, 'P_value': p_value, 'scalar': ci_scalar,
              'Cohen_d': (mean1 - mean2) * np.sqrt(2) /
                          np.sqrt(group1.var(ddof=1) + group2.var(ddof=1)),
              'df': deg_of_freedom
              }
    return output


In [9]:
g = np.random.rand(27) * 50
h = np.random.rand(27) * 55


In [25]:
#z = [0, 2, 3]
z.insert(1, 10)

In [26]:
z

[0, 10, 1, 2, 3]

In [17]:
repeated_ttest(g, h)

{'Mean1': 25.387786060377078,
 'Std1': 14.927211957718175,
 'n1': 27,
 'Mean2': 27.371970570943333,
 'Std2': 18.46482175339166,
 'n2': 27,
 'T_value': -0.4795725661250065,
 'P_value': 0.6355408939090509,
 'scalar': 8.504551680516716,
 'Cohen_d': -0.09229378338274402,
 'df': 26}

In [15]:
repeated_ttest(g, h)

{'Mean1': 25.387786060377078,
 'Std1': 14.927211957718175,
 'n1': 27,
 'Mean2': 27.371970570943333,
 'Std2': 18.46482175339166,
 'n2': 27,
 'T_value': -0.4795725661250065,
 'P_value': 0.6355408939090509,
 'scalar': 8.504551680516716,
 'Cohen_d': -0.09229378338274402,
 'df': 26}

In [3]:
basic_list = np.array([26, 22, 13, 13, 16, 9, 10, 4, 9, 17, 19, 11, 17, 23, 25])
rehearse_list = np.array([11, 11, 10, 6, 5, 7, 6, 9, 6, 10, 5, 13, 24, 23, 27])
delay_list = np.array([24, 13, 13, 15, 9, 6, 6, 4, 3, 10, 8, 9, 10, 5, 8])

In [8]:
p = figure(plot_width=400, plot_height=400)
p.line(range(15), basic_list)
p.line(range(15), rehearse_list, color='red')
p.line(range(15), delay_list, color='black')

show(p)

In [10]:
from scipy.optimize import fmin
fmin?

In [36]:
def _fit_func(A, ):
    t = np.arange(15)
    g1 = A[0] * np.exp(-A[1] * t)
    g2 = A[2] * np.exp(A[3] * t)
    return (np.square(g1+g2 - basic_list).sum() + 
            np.square(g1 - delay_list).sum() + 
            np.square(g2 - rehearse_list).sum())

A = fmin(_fit_func, np.random.rand(4))
    

Optimization terminated successfully.
         Current function value: 824.530276
         Iterations: 423
         Function evaluations: 697


In [137]:
p = figure(plot_width=800, plot_height=500)
p.line(range(15), rehearse_list/29, color='black', line_width=2, line_dash='dotdash', legend='Forced 3x rehearsal (Test3)')
p.line(range(15), delay_list/29, color='black', line_width=2, line_dash='dashed', legend='30s retention interval w/counting (Test 2)')
p.line(range(15), basic_list/29, color='black', line_width=2, legend='No rehearsal contraints, 0s retention interval (Test 1)')
p.circle(range(15), basic_list/29, color='black', size=6)
p.square(range(15), rehearse_list/29, color='black', size=6)
p.diamond(range(15), delay_list/29, color='black', size=8)
# g1 = A[0] * np.exp(-A[1] * np.arange(15))
# g2 = A[2] * np.exp(A[3] * np.arange(15))
# p.line(range(15), g1 )
# p.line(range(15), g2)
# p.line(range(15), g1 + g2)

p.xaxis.ticker = list(range(0, 16, 3))
p.legend.location = 'top_center'
p.legend.label_text_font_size= '12pt'
p.legend.label_text_font = 'Helvetica'

p.xaxis.axis_label = 'Word Position'
p.xaxis.axis_label_text_font_size = '12pt'
p.xgrid.grid_line_color = None
p.yaxis.axis_label = 'Probability of Recall'
p.yaxis.axis_label_text_font_size = '12pt'
p.yaxis.minor_tick_out = None
p.xaxis.major_label_text_font = 'Helvetica'
p.xaxis.major_label_text_font_style = 'normal'
p.yaxis.major_label_text_font = 'Helvetica'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

show(p)

In [1]:
import numpy as np
from scipy import stats

In [9]:
x = [.5, .75, 1., 1.25, 1.5, 1.75, 1.75, 2., 2.25, 2.50, 2.75, 3.0, 3.25, 3.5, 4.0, 4.25, 4.5, 4.75, 5., 5.50]
y = [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
y1 = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]

x = np.array(x)
y = np.array(y)
y1 = np.array(y1)

In [10]:
from sklearn import linear_model

In [11]:
log_model = linear_model.LogisticRegression()
log_model.fit(X=np.expand_dims(x, 1),y=y)

/apps/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [12]:
a, b = log_model.coef_, log_model.intercept_
a = [1.5046]
b = [-4.0777]

In [13]:
p = figure(plot_width=600, plot_height=400)
p.circle(x, y)
p.line(np.linspace(0, 6, 1000), 1 /(1 + np.exp(-a[0]*np.linspace(0, 6, 1000) - b[0])))
show(p)

In [9]:
1 /(1 + np.exp(-a[0]*3 - b[0]))

0.6073293474722868

In [10]:
def sigmoid(a, b, x):
    return 1. / (1.0 + np.exp(-(a * x + b)))

def _min_fun(g, x, y):
    a, b = g
    ht = sigmoid(a, b, x)
    hht = 1.0-ht
    mask = y==1
    return -(np.sum(np.log(ht[mask])) + np.sum(np.log(hht[~mask])))

In [11]:
from scipy.optimize import minimize, basinhopping

In [14]:
_min_fun([a, b], x, y)

8.029878481078779

In [15]:
x0 = [0.5, 0.5]
v = minimize(_min_fun, x0, args=(x, y), method='SLSQP')

In [140]:
import numpy as np
from scipy.optimize import minimize, minimize_scalar
tiny = np.finfo(float).tiny


# Parameter models are variants of sigmoid functions
def one_parameter_model(difficulty, thetas):
    """Items only vary on difficulty."""
    return 1.0 / (1.0 + np.exp(-1 * (thetas - difficulty)))


def two_parameter_model(difficulty, discrimination, thetas):
    """Items vary with difficulty and discrimination."""
    return 1.0 / (1.0 + np.exp(-discrimination * (thetas - difficulty)))


def three_parameter_model(difficulty, discrimination, guessing, thetas):
    """Items vary with difficulty, discrimination and guessing."""
    return (guessing + (1.0 - guessing) /
                       (1.0 + np.exp(-discrimination * (thetas - difficulty))))


def get_quadrature_points(nPoints, start, stop):
    """Returns the quadrature points to aid in integration.
    
    Args:
        nPoints:  Number of points to use
        start: beginning of the interval
        stop: end of the interval
        
    Returns:
        (positions, weights, scalar)
    
    """
    # Get the knots and weights
    x_position, weights = np.polynomial.legendre.leggauss(nPoints)
    
    # convert to appropriate interval
    scalar = 0.5 * (stop - start)
    x_position = (x_position + 1) * scalar + start

    return (x_position, weights, scalar)
    
    
irt_model = {1: one_parameter_model,
             2: two_parameter_model,
             3: three_parameter_model}


def _item_min_func(params, thetas, dependent_var, sigmoid_model):
    """Minimum function for irt (e-step)."""
    probability_one = sigmoid_model(*params, thetas)
    probability_zero = 1. - probability_one
    mask = dependent_var == 1

    # Return negative since this is going into a minimization function
    return (np.sum(np.log(probability_one[mask] + tiny)) +
            np.sum(np.log(probability_zero[~mask] + tiny))) * -1


def item_parameter_solver(thetas, dependent_y, model=2, initial_guess=None):
    """Univariate Item Response Theory solver.

    Solves for sigmoid parameters, given by model, for an item.

    Inputs must be numpy arrays

    Args:
        thetas: Input Abilities of respondents
        dependent_y:  Item level measured responses (must be 0 or 1)
        model: [1, (2), 3] Number of parameters to use in model estimation
        initial_guess:  Seeding value of parameters, if None then random
                        values are used.  Must be length of model if supplied

    Returns:
        computed parameters based on maximum likelihood
    """
    sigmoid_model = irt_model[model]
    if initial_guess is None:
        initial_guess = np.random.randn(model)

    # Perform the minimization
    results = minimize(_item_min_func, initial_guess, method='SLSQP',
                       args=(thetas, dependent_y, sigmoid_model))

    if not results.success:
        warnings.warn("Convergence didn't occur, returning last solution")

    return results.x


def _theta_min_func(theta, dependent_var, params, sigmoid_model):
    """minimization function for ability, marginal maximum likelihood."""
    probability_one = np.zeros((params.shape[0], ))

    for ndx in range(params.shape[0]):
        probability_one[ndx] = sigmoid_model(*params[ndx], theta)

    probability_zero = 1 - probability_one
    mask = dependent_var == 1
    
    # Return negative since this is going into a minimization function
    return (np.sum(np.log(probability_one[mask] + tiny)) +
            np.sum(np.log(probability_zero[~mask] + tiny))) * -1


def theta_parameter_solver(item_params, dependent_y):
    """Solves for ability given a list of item parameters.

    Given a set of sigmoid parameters, solve for the optimal theta using
    marginal maximum likelihood

    Args:
        item_params: (n_items x model) matrix of item parameters
        dependent_y:  Person level measured responses (must be 0 or 1)

    Returns:
        ability for the person
    """
    model = item_params.shape[1]
    sigmoid_model = irt_model[model]

    results = minimize_scalar(_theta_min_func, bracket=[-4, 4], bounds=(-4, 4),
                              args=(dependent_y, item_params, sigmoid_model), method='bounded')

    return results



In [141]:
xp = (x - x.mean()) / x.std(ddof=1)
#xp = np.random.rand
g = item_parameter_solver(xp, y)
g1 = item_parameter_solver(xp, y1)


In [142]:
g, g1

(array([-0.05134159,  2.26783281]), array([-0.08419978,  3.88146587]))

In [143]:
p = figure(plot_width=600, plot_height=400)
p.circle(xp, y)
p.line(np.linspace(-3, 3, 1000), 1 /(1 + np.exp(-g[1]*(np.linspace(-3, 3, 1000) - g[0]))))
p.circle(xp, y1, fill_color='red')
p.line(np.linspace(-3, 3, 1000), 1 /(1 + np.exp(-g1[1]*(np.linspace(-3, 3, 1000) - g1[0]))), line_color='red')

show(p)

In [109]:
theta_parameter_solver(np.vstack((g, g1)), np.array([y[8], y1[8]]))

     fun: 1.3653670066632775
 message: 'Solution found.'
    nfev: 12
  status: 0
 success: True
       x: -0.2395712522299039

In [110]:
xp[8]

-0.3566298538452465

In [96]:
_func = irt_model[2]

In [106]:
for t in np.linspace(-3,3, 40):
    print(t, (1-_func(*g, t))*_func(*g1, t))

-3.0 1.2159246759597039e-05
-2.8461538461538463 2.2079137030177176e-05
-2.6923076923076925 4.008298056150797e-05
-2.5384615384615383 7.274421828418784e-05
-2.3846153846153846 0.00013195842547867717
-2.230769230769231 0.0002392151309387286
-2.0769230769230766 0.0004332377741848468
-1.923076923076923 0.0007835472919864616
-1.7692307692307692 0.001414275225137703
-1.6153846153846154 0.0025452562522863815
-1.4615384615384615 0.004561013615918048
-1.3076923076923075 0.008121371262168414
-1.1538461538461537 0.014324730003786685
-1.0 0.0249108342790376
-0.8461538461538458 0.04240731687968987
-0.6923076923076921 0.0699234866888538
-0.5384615384615383 0.10997230020399523
-0.3846153846153846 0.1616514207944187
-0.23076923076923084 0.21699011909861116
-0.07692307692307665 0.26081378355882323
0.0769230769230771 0.27857163759575676
0.23076923076923084 0.2666088868710947
0.384615384615385 0.23320494113057427
0.5384615384615388 0.19073739791131952
0.6923076923076925 0.14879709916174083
0.846153846153

In [112]:
x, w = np.polynomial.legendre.leggauss(6)

In [2]:
from bokeh.models import ColumnDataSource
from bokeh.models.glyphs import Ellipse
from bokeh.layouts import Row

In [127]:
from scipy import integrate

# Define function and interval
a = -np.pi
b = np.pi
f = lambda x: np.cos(x)

# Gauss-Legendre (default interval is [-1, 1])
points, weights, scalar = get_quadrature_points(6, a, b)
gauss = sum(weights * f(points)) * scalar

# For comparison
quad, quad_err = integrate.quad(f, a, b)

print ('The QUADPACK solution: {0:.12} with error: {1:.12}'.format(quad, quad_err))
print ('Gauss-Legendre solution: {0:.12}'.format(gauss))
print ('Difference between QUADPACK and Gauss-Legendre: ', abs(gauss - quad))

The QUADPACK solution: 2.61590137351e-16 with error: 4.47173709324e-14
Gauss-Legendre solution: -3.75133023187e-06
Difference between QUADPACK and Gauss-Legendre:  3.751330232135064e-06


In [25]:
pt1 = figure(plot_width=400, plot_height=400, x_range=[-1, 1], y_range=[-2, 2], tools=[], title='No Delay, p=0.673')
x = np.random.randn(90)*0.2
y = np.random.randn(90)*.5
pt1.circle(x, y, fill_color='blue')
pt1.oval(0, 0, 2.5*.2, 2.5*.5, fill_alpha=.3)
pt1.toolbar.logo = None


In [26]:
pt2 = figure(plot_width=400, plot_height=400, x_range=[-1, 1], y_range=[-2, 2], tools=[], title='Delay, Red-Red, p=0.461')
x = np.random.randn(30)*0.21
y = np.random.randn(30)*.55
pt2.circle(x, y, fill_color='red')
pt2.oval(0, 0, 2.5*.21, 2.5*.55, angle=0, fill_alpha=.3, fill_color='red', line_color='red')
pt2.toolbar.logo = None

# Correlation for Red/Blue
pt3 = figure(plot_width=400, plot_height=400, x_range=[-1, 1], y_range=[-2, 2], tools=[], title='Delay, Red-Blue, p=.454')
angle = np.pi/10 
corRB = np.array([[np.cos(angle), np.sin(angle)], [-np.sin(angle), np.cos(angle)]])
nb = corRB @ np.array([x*.22/.21, y*.92/.55])
x1 = nb[0]
y1 = nb[1]
pt3.circle(x1, y1, fill_color='purple')
pt3.oval(0, 0, 2*.25, 2*.95, angle=-np.pi/10, fill_alpha=.3, fill_color='purple', line_color='purple')
pt3.toolbar.logo = None

pt4 = figure(plot_width=400, plot_height=400, x_range=[-1, 1], y_range=[-2, 2], tools=[], title='Delay, Green-Purple, p=.244')
angle = np.pi/20 
corRB = np.array([[np.cos(angle), np.sin(angle)], [-np.sin(angle), np.cos(angle)]])
nb = corRB @ np.array([x*.22/.2, y*.92/.5])
x1 = nb[0]
y1 = nb[1]
pt4.circle(x1, y1, fill_color='grey')
pt4.oval(0, 0, 2*.25, 2*.95, angle=-np.pi/20, fill_alpha=.3, fill_color='grey', line_color='grey')
pt4.toolbar.logo = None


show(Row(pt1, pt2, pt3, pt4))

In [71]:
np.array([x*.25/.2, y*.95/.5]).shape

(2, 90)

In [32]:
import pandas

In [33]:
ff = "/home/ryan/Downloads/Collaborative Inhibition Data winter 2019.csv"
data = pandas.read_csv(ff)

In [34]:
data

,Type,Dyad,Words,Categories,Errors
0,1,1,43,13,2
1,1,2,48,12,4
2,1,3,47,14,1
3,1,4,26,9,3
4,1,5,31,9,0
5,2,6,68,14,11
6,2,7,53,13,5
7,2,8,39,12,2
8,2,9,40,12,2
9,2,10,65,14,2


In [52]:
condition = data['Type'].values
words = data['Words'].values
cats = data['Categories'].values
errs = data['Errors'].values

In [53]:
stats.ttest_ind(words[condition==2], words[condition==1], equal_var=False)

Ttest_indResult(statistic=1.864182744519576, pvalue=0.10264827562037836)

In [54]:
stats.ttest_ind(cats[condition==2], cats[condition==1], equal_var=False)

Ttest_indResult(statistic=1.4253932901995963, pvalue=0.2086400253197752)

In [55]:
stats.ttest_ind(errs[condition==2], errs[condition==1], equal_var=False)

Ttest_indResult(statistic=1.2719974560076321, pvalue=0.2565830639475785)

In [56]:
words[condition==2].mean(), words[condition==1].mean()

(53.0, 39.0)

In [58]:
words[condition==2].std(ddof=1), words[condition==1].std(ddof=1)

(13.546217184144066, 9.924716620639604)

In [63]:
cats[condition==2].mean(), cats[condition==1].mean()

(13.0, 11.4)

In [59]:
cats[condition==2].std(ddof=1), cats[condition==1].std(ddof=1)

(1.0, 2.3021728866442674)

In [64]:
errs[condition==2].mean(), errs[condition==1].mean()

(4.4, 2.0)

In [60]:
errs[condition==2].std(ddof=1), errs[condition==1].std(ddof=1)

(3.9115214431215892, 1.5811388300841898)

In [61]:
from factoranalysis.response_utils import 

In [65]:
def cohen_d(sample1, sample2):
    """Computes the standardized distance between two samples.

    Args:
        sample1:  First sample to compare
        sample2:  Second sample to compare

    Returns:
        distance in pooled standard deviation units
    """
    denominator = np.sqrt(sample1.var() + sample2.var())
    return (sample1.mean() - sample2.mean()) / denominator

In [67]:
cohen_d(words[condition==2], words[condition==1])

0.932091372259788

In [68]:
cohen_d(cats[condition==2], cats[condition==1])

0.7126966450997981

In [69]:
cohen_d(errs[condition==2], errs[condition==1])

0.6359987280038161

In [1]:
import numpy as np

In [21]:
g = np.random.randn(3, 5)
h = np.random.randn(1, 5)
g[2, 4] = np.nan
g[1, 3] = np.nan
h[0, 2] = np.nan

In [22]:
g

array([[-1.9663834 ,  0.10896951, -0.76973536,  0.86452387,  2.17771652],
       [ 0.15856995, -0.73567037, -2.99423456,         nan, -1.08963546],
       [ 1.06812496, -0.69134385,  0.41690671, -0.26614177,         nan]])

In [34]:
np.count_nonzero((np.count_nonzero(np.isnan(g), axis=0) == 0) & ~np.isnan(h))

2

In [45]:
np.unique(np.random.choice(1000, size=1500, replace=False)).shape


ValueError: Cannot take a larger sample than population when 'replace=False'

In [50]:
g[slice(None, None)].tolist()

[[-1.966383397690178,
  0.10896951028633803,
  -0.7697353581155413,
  0.8645238691272116,
  2.1777165217618264],
 [0.1585699484447738,
  -0.7356703679140354,
  -2.994234562989829,
  nan,
  -1.0896354643343662],
 [1.0681249553008874,
  -0.6913438513454357,
  0.41690670765652094,
  -0.266141765310602,
  nan]]

In [58]:
np.hypot?

In [1102]:
def thurstone26_box(n_size=1000):
    """Creates the thursone 26 box problem for factor analysis.
    
    Args:
        n_size:  Number of realizations
    
    Returns:
        An ndarray of size [26 x n_size]
    """
    x1 = np.random.randn(n_size) * .2 + 10
    x2 = np.random.randn(n_size) * .2 + 10
    x3 = np.random.randn(n_size) * .2 + 10
    
    output = np.zeros((26, n_size))
    
    # fill the data
    output[0] = x1.copy()
    output[1] = x2.copy()
    output[2] = x3.copy()
    output[3] = x1 * x2
    output[4] = x1 * x3
    output[5] = x2 * x3
    output[6] = x1 * x1 * x2
    output[7] = x1 * x2 * x2
    output[8] = x1 * x1 * x3
    output[9] = x1 * x3 * x3
    output[10] = x2 * x2 * x3
    output[11] = x2 * x3 * x3
    output[12] = x1 / (x2 + 1e-23)
    output[13] = x2 / (x1 + 1e-23)
    output[14] = x1 / (x3 + 1e-23)
    output[15] = x3 / (x1 + 1e-23)
    output[16] = x2 / (x3 + 1e-23)
    output[17] = x3 / (x2 + 1e-23)
    output[18] = 2 * (x1 + x2)   
    output[19] = 2 * (x1 + x3)
    output[20] = 2 * (x2 + x3)
    output[21] = np.hypot(x1, x2)
    output[22] = np.hypot(x1, x3)
    output[23] = np.hypot(x2, x3)
    output[24] = x1 * x2 * x3
    output[25] = np.sqrt(x1*x1 + x2*x2 + x3*x3)

    return output

In [714]:
from irt_source import IRTArray
from irt_source import rasch_model, one_parameter_model, two_parameter_model

In [2]:
import numpy as np
import scipy as sp
from scipy import integrate

In [211]:
g = IRTArray([('Abilities', 15), ('Difficulty', np.random.randn(5)), ('Discrimination', np.random.randn(1))])
k = np.array([1, 0, 0, 1])[:, None]

In [239]:
gaus = lambda x : 1 / np.sqrt(2 * np.pi) * np.exp(-np.square(x) / 2)

def local_func(ability):
    g['Abilities'] = ability
    temp = one_parameter_model(g)
    temp *= gaus(ability)[None, :]
    return temp

integrate.fixed_quad(local_func, -6, 6, n=15)

(IRTArray([0.42958082, 0.48360689, 0.5296868 , 0.47141115, 0.49877639]), None)

In [229]:
alpha  = 1 / 1.7 / discrimination
gamma = np.sqrt(1 + np.pi / np.square(alpha) / 8)

In [230]:
the_int = lambda dif : 0.5 + 0.5 * np.erf(-1 * np.sqrt(np.pi) / 4 / gamma * dif)
the_int2 = lambda dif : 1 / (1 + np.exp(dif / gamma / alpha))

In [182]:
g['Difficulty']

IRTArray([ 1.39654805, -0.59773102,  0.01331596,  0.4377436 , -0.39657728])

In [195]:
the_int(g['Difficulty'])

IRTArray([0.80845884, 0.54346301, 0.54332677, 0.19151238, 0.24766022])

In [231]:
the_int2(g['Difficulty'])

IRTArray([0.4295191 , 0.48358471, 0.5297002 , 0.47137964, 0.49876596])

In [235]:
theta = np.linspace(-8, 8, 100001)
difficulty = g['Difficulty'][1]
discrimination = g['Discrimination'][0]
test = IRTArray([('Abilities', theta), ('Difficulty', np.array([difficulty])), 
                 ('Discrimination', np.array([discrimination]))])
probs = one_parameter_model(test)
dist = 1 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2) * 16 / 100000

In [236]:
(probs * dist).sum()

IRTArray(0.48359775)

In [237]:
the_int2(difficulty)

0.48358471038762807

In [ ]:
def marginal_integral():
    pass

In [3]:
theta1 = np.linspace(-5, 5, 101)
theta2 = np.linspace(-5, 5, 101)
theta3 = np.linspace(-5, 5, 101)
t1, t2, t3 = np.meshgrid(theta1, theta2, theta3)
a = 2.3
b = 1.7*2.3
c = 0
d =0
pfunc = 1 / (1 + np.exp(-a*t1 - b*t2 - c*t3 + d))
gaust = np.multiply.outer(np.outer(gaus(theta1), gaus(theta2)) / 100, gaus(theta3)/10)
print((pfunc * gaust).sum())




NameError: name 'gaus' is not defined

In [273]:
denom = np.sqrt(1 + np.pi / 8 * (a*a + b*b + c*c))
print(1 / (1 + np.exp(d / denom)))

0.5


In [1867]:
from scipy.optimize import basinhopping
n_items = 50
n_takers = 1000
abilities = np.random.randn(n_takers)
difficulty =  np.linspace(-2., 2., n_items)
discrimination = np.array([1.24]) #0.3*np.random.randn(n_items) + 1.5

irt_true = IRTArray([('Abilities', abilities), ('Difficulty', difficulty), ('Discrimination', discrimination)])
parms = one_parameter_model(irt_true)
dataset = np.random.rand(n_items, n_takers) <= parms
#dataset = parms.round(0)
gaus = lambda x : 1 / np.sqrt(2 * np.pi) * np.exp(-np.square(x) / 2)


In [1879]:
for nd in range(dataset.shape[0]):
    g = (abilities[dataset[nd]].mean() * np.count_nonzero(dataset[nd])) - (abilities[dataset[nd]==0].mean() * np.count_nonzero(dataset[nd]==0))
    print(g / (np.count_nonzero(dataset[nd]) - np.count_nonzero(dataset[nd]==0)))

0.3008158466927284
0.32274030488495437
0.3890151327375251
0.3360037287206411
0.44346402477269536
0.41418176537477036
0.4595243613259974
0.5262871487039099
0.6168910820787186
0.5524140516647502
0.6964270023941935
0.7088821117384345
0.918899141430207
0.7955686376206274
1.1085618735641756
1.1219970133747899
1.2737808177361034
1.5180396770219804
1.5660249929914407
2.2811705796687916
2.592566566769227
3.2913825719581626
3.6469164434023082
8.821077721241043
11.465352934194998
-32.57006394489697
-22.155588798754934
-10.693512457845895
-4.839075268263346
-2.1040182782718935
-2.1870487073238825
-1.9946520702452493
-1.2231832232621076
-1.4253058103659242
-1.3760829498215934
-1.0588408310776176
-0.8978086101976698
-0.8933752160214025
-0.6558612169363974
-0.7080382377611483
-0.5384489745198612
-0.5997951515326974
-0.5042983023740798
-0.44403509402386493
-0.4515107574204448
-0.49349002759006966
-0.36528892277928154
-0.34345892310439996
-0.3028089713377923
-0.28034004924092143


In [1873]:
difficulty[0]

-2.0

In [1814]:
dataset = np.fromfile('/home/ryan/Downloads/IRT tutorial/ouirt.dat',
                      dtype='int', sep=" ").reshape(500, 10).transpose()

In [1868]:
yess = np.count_nonzero(dataset, axis=1)
nooo = dataset.shape[1] - yess

In [691]:
approx_int = lambda x, y: 1 / (1 + np.exp(x*discrimination  / np.sqrt(1 + np.pi*discrimination*discrimination /8)))

In [1921]:
def min_func(guess):
    probss = approx_int(guess[:-1], guess[-1]).clip(1e-23, 1 - 1e-23)
    return -(yess * np.log(probss) + nooo * np.log(1 - probss)).sum()

def local_func(ability, g):
    g['Abilities'] = ability
    temp = one_parameter_model(g)
    temp *= gaus(ability)[None, :]
    return temp
    
def min_func2(guess):
    k = IRTArray([('Abilities', 21), ('Difficulty', nooo.size), ('Discrimination', np.array([guess]))])
    k['Difficulty'] = np.sqrt(1 + np.pi * guess * guess / 8) * np.log(nooo / yess) / guess
    probbs = integrate.fixed_quad(local_func, -3, 3, (k,), n=21)[0].clip(1e-23, .99999999)
    print(probbs)
    return -(yess * np.log(probbs) + nooo * np.log(1 - probbs)).sum()



In [1914]:
x = np.pi * discrimination**2 / 8
alpha = 1/discrimination
beta = np.sqrt((1 + alpha * alpha * x) / (alpha*alpha * (1 + x)))
guess = IRTArray([('Difficulty', difficulty), ('Discrimination', discrimination)])
guess2 = IRTArray([('Difficulty', difficulty*beta), ('Discrimination', np.ones((1,)))])
min_func(guess), min_func(guess2)

(IRTArray(28276.34609555), IRTArray(28397.39319713))

array([-2.        , -1.91836735, -1.83673469, -1.75510204, -1.67346939,
       -1.59183673, -1.51020408, -1.42857143, -1.34693878, -1.26530612,
       -1.18367347, -1.10204082, -1.02040816, -0.93877551, -0.85714286,
       -0.7755102 , -0.69387755, -0.6122449 , -0.53061224, -0.44897959,
       -0.36734694, -0.28571429, -0.20408163, -0.12244898, -0.04081633,
        0.04081633,  0.12244898,  0.20408163,  0.28571429,  0.36734694,
        0.44897959,  0.53061224,  0.6122449 ,  0.69387755,  0.7755102 ,
        0.85714286,  0.93877551,  1.02040816,  1.10204082,  1.18367347,
        1.26530612,  1.34693878,  1.42857143,  1.51020408,  1.59183673,
        1.67346939,  1.75510204,  1.83673469,  1.91836735,  2.        ])

In [1931]:
np.log(1 + np.exp(1 + np.exp(3 + np.sqrt(1 + np.pi / 8)*dr[0]))) - np.log(1 + np.exp(1 + np.exp(-3 + np.sqrt(1 + np.pi / 8)*dr[0])))

1.5100229599307664

In [1922]:
min_func2(1.), min_func2(1.2)

[0.88691702 0.8774964  0.85555585 0.86703938 0.84097811 0.83786055
 0.83370746 0.79345282 0.7760479  0.78525139 0.74351953 0.76890648
 0.71630001 0.74554382 0.68229094 0.68827242 0.65448062 0.63964618
 0.63964618 0.59340216 0.57772577 0.56500767 0.56305236 0.52400111
 0.51814975 0.49182573 0.48890089 0.47915046 0.4498795  0.39604874
 0.39997438 0.38524239 0.34182912 0.34875836 0.35666634 0.31500926
 0.29503814 0.29703955 0.22941273 0.25276867 0.22635482 0.23348578
 0.19562662 0.1863553  0.18532366 0.18841767 0.16463035 0.14903899
 0.12712521 0.1344393 ]
[0.88773503 0.87821375 0.85605195 0.86764863 0.84134029 0.83819559
 0.83400721 0.79346265 0.77596161 0.78521376 0.74329851 0.7687857
 0.71600783 0.74532956 0.68195684 0.68794223 0.65414579 0.63932159
 0.63932159 0.59314812 0.57750629 0.56481923 0.5628689  0.52392582
 0.51809166 0.49184614 0.48893003 0.47920855 0.45002142 0.39631831
 0.40023634 0.38553115 0.34216439 0.34908992 0.35699182 0.31534336
 0.29535356 0.29735759 0.2295382  0.252

(IRTArray(28245.17557774), IRTArray(28245.27811795))

In [1902]:
v = np.concatenate((np.sqrt(1 + np.pi/8) *np.log(nooo / yess),))

In [1903]:
minimizer_kwargs = {'method': 'SLSQP'}
result = basinhopping(min_func2, [1.0], niter_success=10,
                      minimizer_kwargs=minimizer_kwargs)

ValueError: all the input arrays must have same number of dimensions

In [1607]:
dr = result.x / np.sqrt(1 + np.pi / 8)

In [750]:
gamma2 = np.sqrt(1 + np.pi / 8)
(gamma2 * np.log(nooo/yess))

array([2.04704906, 1.1857837 , 0.90041105, 1.0224841 , 0.29418546,
       0.92225091, 0.41977532, 0.8144629 , 0.90041105, 3.05262122])

In [738]:
2.03374693 / 1.9333

1.0519562044173174

In [749]:
 min_func2(result.x)

IRTArray(2869.00323776)

In [744]:
 min_func2(result.x)

IRTArray(2869.00323776)

In [454]:
min_func(result.x), min_func2(result.x)

(IRTArray(335879.80475343), IRTArray(155330.17192098))

In [552]:
list(zip(result.x[:-1]/discrimination, difficulty))

[(-2.346176735179492, -2.5),
 (-2.3505982932659495, -2.4494949494949494),
 (-2.1942053915259914, -2.398989898989899),
 (-2.1675672846197545, -2.3484848484848486),
 (-2.1941997508137683, -2.297979797979798),
 (-2.1600740923733426, -2.2474747474747474),
 (-2.049557497693188, -2.196969696969697),
 (-2.0293085873361214, -2.1464646464646466),
 (-2.022632208631022, -2.095959595959596),
 (-1.9204889910638572, -2.0454545454545454),
 (-1.8753696862538798, -1.9949494949494948),
 (-1.8035791400382655, -1.9444444444444444),
 (-1.828953954063634, -1.893939393939394),
 (-1.756948040108078, -1.8434343434343434),
 (-1.6558381165009284, -1.7929292929292928),
 (-1.6043740836414275, -1.7424242424242424),
 (-1.585263868288697, -1.691919191919192),
 (-1.5248880148579174, -1.6414141414141414),
 (-1.462601006757598, -1.5909090909090908),
 (-1.466961430661392, -1.5404040404040404),
 (-1.4867508430752758, -1.4898989898989898),
 (-1.373827595259801, -1.4393939393939394),
 (-1.313417076194229, -1.388888888888888

In [588]:
result.x

array([-2.90925915, -2.91474188, -2.72081469, -2.68778343, -2.72080769,
       -2.67849187, -2.5414513 , -2.51634265, -2.50806394, -2.38140635,
       -2.32545841, -2.23643813, -2.2679029 , -2.17861557, -2.05323926,
       -1.98942386, -1.9657272 , -1.89086114, -1.81362525, -1.81903217,
       -1.84357105, -1.70354622, -1.62863717, -1.51438263, -1.51206524,
       -1.39508589, -1.36894398, -1.29650274, -1.18377668, -1.16183645,
       -1.1145842 , -1.03762799, -1.10095119, -0.95713983, -0.83041915,
       -0.7357355 , -0.76354273, -0.65516984, -0.67217902, -0.54278798,
       -0.61460781, -0.49491391, -0.42953161, -0.33743423, -0.29099015,
       -0.18605876, -0.21141486, -0.20031786, -0.06610062, -0.02517775,
       -0.01416231,  0.08500653,  0.11022001,  0.16864718,  0.20189988,
        0.29098623,  0.3760721 ,  0.45396301,  0.46376642,  0.44091787,
        0.55108252,  0.73054543,  0.71673461,  0.84287498,  0.8949739 ,
        0.88052077,  0.87331585,  1.02626109,  1.13810031,  1.12

In [705]:
gamma = np.sqrt(1 + np.pi * 1.3782**2 / 8) / 1.3782

In [706]:
gamma*np.log(nooo / yess)

array([1.66302144, 0.96332997, 0.7314934 , 0.83066548, 0.23899609,
       0.74923609, 0.34102522, 0.66166917, 0.7314934 , 2.47994767])

In [702]:
2.0470 / 1.6581

1.234545564199988

In [664]:
x = np.pi * discrimination**2 / 8
alpha = 1/discrimination
beta = np.sqrt((1 + alpha * alpha * x) / (alpha*alpha * (1 + x)))

In [668]:
gamma * np.log(nooo / yess) / discrimination

IRTArray([-1.92054676, -2.01567304, -1.90580064, -1.94447144, -1.79578759,
          -1.83709944, -1.72426513, -1.79307514, -1.67592875, -1.65603153,
          -1.59072238, -1.52802803, -1.44155586, -1.43078869, -1.40099953,
          -1.36757596, -1.39048509, -1.33681198, -1.23982956, -1.22446683,
          -1.13476729, -1.12565243, -1.14943711, -1.03838159, -0.94943441,
          -0.92772339, -0.97134761, -0.8395411 , -0.84114662, -0.84436062,
          -0.82194454, -0.67894295, -0.72019484, -0.60094482, -0.62176873,
          -0.5304331 , -0.49564519, -0.55232817, -0.44826395, -0.41268514,
          -0.35485621, -0.31429551, -0.26012773, -0.26151179, -0.21733303,
          -0.15827782, -0.12953865, -0.12270395, -0.05175718,  0.01634122,
           0.04630708,  0.04630708,  0.13364084,  0.15964783,  0.19807074,
           0.19807074,  0.2836902 ,  0.32964998,  0.35064794,  0.40135288,
           0.39993808,  0.48267311,  0.50865661,  0.52606851,  0.58319204,
           0.65324289,  0

In [669]:
result.x[:-1]

array([-1.92055279, -2.01566781, -1.90580094, -1.94447172, -1.79579208,
       -1.83709892, -1.72426474, -1.79308221, -1.67592444, -1.65603224,
       -1.59072325, -1.52803008, -1.44155654, -1.43079568, -1.40100569,
       -1.3675734 , -1.39048445, -1.33681144, -1.23983379, -1.22446375,
       -1.13476703, -1.12565044, -1.14943631, -1.03838069, -0.94943384,
       -0.92772415, -0.97134854, -0.83954091, -0.84114509, -0.84436417,
       -0.82194012, -0.6789382 , -0.72019273, -0.60094378, -0.62176897,
       -0.53043206, -0.49564253, -0.5523299 , -0.44826193, -0.41268511,
       -0.3548571 , -0.31429478, -0.26012673, -0.26151092, -0.21733225,
       -0.15827702, -0.12953848, -0.12270456, -0.0517582 ,  0.01633965,
        0.04630611,  0.04631482,  0.13363967,  0.15964265,  0.19807235,
        0.19807045,  0.28369057,  0.32964812,  0.35064796,  0.40135309,
        0.39993722,  0.48267225,  0.50865596,  0.52607286,  0.58318834,
        0.65324246,  0.68958382,  0.69264186,  0.69415842,  0.76

In [707]:
discrimination = np.array([1.4183, 1.1445, 1.191, 1.6444, 1.3398, 1.4973, 0.8478, 1.5133, 1.8716, 2.1060])

In [709]:
gamma = np.sqrt(1 + np.pi * discrimination**2 / 8) / discrimination

In [712]:
gamma * np.log(nooo / yess)

array([1.63625572, 1.08038793, 0.79937353, 0.756575  , 0.24294311,
       0.71570847, 0.47509735, 0.62851505, 0.6283245 , 2.03374693])

In [756]:
dataset[0,0]

0

In [801]:
abilities

array([ 1.0028401 ,  0.77424018,  0.58839102, ..., -0.07468689,
        0.04069331, -1.31097752])

In [1363]:
a = np.linspace(0, 4, 1000)
tes = lambda x, y, ndx : 1 / (1 + np.exp((-1)**(dataset[ndx, 2]) * x * (y - np.sqrt(1 + np.pi * x * x / 8)/x*np.log(nooo[ndx]/yess[ndx]))))

In [1368]:
ax = np.linspace(-4, 4, 100)
axo = np.zeros_like(ax)
for ndd, abil in enumerate(ax):
    v = np.ones(999)
    for ndx in range(dataset.shape[0]):
        v *= tes(a[1:], abil, ndx)
    axo[ndd] = v.sum()

In [1276]:
l = quadrature_func(a[1:], ax[50], dataset[:, 0], dr)

In [1250]:
axo[50]

1.6996770759446413e-60

In [1353]:
v = np.ones(999)
for ndx in range(dataset.shape[0]):
    v *= tes(a[1:], ax[50], ndx)

In [1354]:
p = figure(plot_width=600, plot_height=400)
p.line(a[1:], l)
p.line(a[1:], v, line_color='red')
show(p)


In [1369]:
p = figure(plot_width=600, plot_height=400)
p.line(ax, axo)
show(p)

In [1367]:
otpt[2]

-0.7357502480292115

In [1065]:
def sigmy(theta, mu, a=1):
    return 1 / (1 + np.exp(-a* (theta - mu)))
def gaus(theta):
    return 1 / np.sqrt(2*np.pi) * np.exp(-theta**2 / 2)
    

In [1932]:
sigmy(xx, -.2, 1.2).sum()

7.759343644687297

In [1934]:
np.log(1 + np.exp(1 + np.exp(6 + np.sqrt(1 + np.pi / 8)*dr[0]))) - np.log(1 + np.exp(1 + np.exp(-6 + np.sqrt(1 + np.pi / 8)*dr[0])))

35.146371219480905

In [1192]:
xx = np.linspace(-5, 5, 1000)
yy1 = sigmy(xx, 1.32, 1.3)
yy2 = sigmy(xx, 0.74, -.86)
yy3 = sigmy(xx, -0.34, 1.1)
yy4 = sigmy(xx, 1.55, 1.35)
gg = gaus(xx) / 100


In [1195]:
plt = figure(plot_width=600, plot_height=400)
plt.line(xx, yy1 *yy3)
plt.line(xx, yy1, line_color='red')
plt.line(xx, yy2, line_color='red')
plt.line(xx, yy3, line_color='red')
plt.line(xx, yy4, line_color='black')
# plt.line(xx, yy1*yy3 - yy4)
show(plt)

In [1044]:
sigmy(-1.32 / np.sqrt(1 + .368), 0)

0.24442395620161766

In [1080]:
np.log((gg * yy1 * yy2 * yy3).sum())

-2.6475989456799858

In [1084]:
(np.log(yy1) + np.log(yy2) + np.log(yy3) + np.log(gg)).sum()

-14735.666136672526

In [1045]:
np.log(sigmy((-1.32 - 0.319) / np.sqrt(1 + .205), 0))

-1.6957658416073902

In [918]:
np.sqrt(1 + np.pi * 1.24**2 / 8) * np.log(nooo/ yess) / 1.24

IRTArray([-2.08413107, -1.99360655, -1.95051791, -1.88830852, -1.83709944,
          -1.82596613, -1.82043179, -1.69355612, -1.66843669, -1.63151953,
          -1.60499941, -1.53487353, -1.55096095, -1.49422839, -1.3988915 ,
          -1.39258285, -1.33681198, -1.24368984, -1.26702023, -1.21302569,
          -1.16978545, -1.14943711, -1.07889572, -1.02446568, -1.03489432,
          -1.043623  , -0.90950478, -0.88487769, -0.80446304, -0.7309813 ,
          -0.7823745 , -0.76201892, -0.67439276, -0.6337244 , -0.6382186 ,
          -0.56552465, -0.51735336, -0.5552568 , -0.4411274 , -0.40418363,
          -0.40276807, -0.37594314, -0.24629994, -0.25182829, -0.21320227,
          -0.17747138, -0.15827782, -0.09948558, -0.04085764, -0.06265967,
           0.02315056,  0.03268454,  0.04903204,  0.1637587 ,  0.1418485 ,
           0.22422159,  0.23525393,  0.35766339,  0.40418363,  0.32127034,
           0.39993808,  0.43827576,  0.50142312,  0.48987504,  0.59946208,
           0.60985426,  0

In [1204]:
integrate.fixed_quad?
tes = lambda x, y, ndx : 1 / (1 + np.exp((-1)**(dataset[ndx, 0]) * x * (y - np.sqrt(1 + np.pi * x * x / 8)/x*np.log(nooo[ndx]/yess[ndx]))))

In [1694]:
np.sqrt(2/np.pi)

0.7978845608028654

In [1880]:
dr = np.log(nooo / yess)


def quadrature_func2(theta, discriminate, response, dr):
    the_sign = (-1)**response
    gauss = 1 / np.sqrt(2 * np.pi) * np.exp(-np.square(theta) / 2)
    return  gauss * (1 / (1 + np.exp(discriminate * np.outer(the_sign, theta) - np.outer(dr*the_sign, np.sqrt(1 + 1/8 * np.pi * discriminate**2)) ))).prod(0)


def quadrature_func(discriminate, theta, sigma, response, dr):
    the_sign = (-1)**response
    gauss = 1/np.sqrt(2*np.pi * .125) * np.exp(-np.square(discriminate - sigma) / 2 / .125)
    return gauss * (1 / (1 + np.exp(theta * np.outer(the_sign, discriminate) - np.outer(dr*the_sign, np.sqrt(1 + 1/8 * np.pi * discriminate**2)) ))).prod(0)
    
def discriminate_func(alpha, thetas, response, dr):
    the_sign = (-1)**response
    return (1 / (1 + np.exp(alpha * the_sign * thetas - (the_sign * dr[:, None]* np.sqrt(1 + np.pi * alpha * alpha /8)))))
    
def min_func(theta, ndx):
    return -integrate.fixed_quad(quadrature_func, a[1], 4, (theta, dataset[:, ndx], dr), n=81)[0]

dataset2, counts = np.unique(dataset, axis=1, return_counts=True)
def min_func_try(discrim):
    otpt = list(map(lambda x: integrate.fixed_quad(quadrature_func2, -4, 4, 
                                                   (discrim, dataset2[:, x], dr), n=81)[0], range(dataset2.shape[1])))
    return -np.log(otpt).dot(counts)

In [1881]:
discr_guess = optimize.fminbound(min_func_try, a[1], 4)
discr_guess

1.223679763684509

In [1857]:
discr_guess = optimize.fminbound(min_func_try, a[1], 4)
discr_guess

1.3725827336511025

In [1804]:
discrimination

array([1.24])

In [1563]:
dr * np.sqrt(1 + np.pi * 1.83**2 / 8)/1.83

IRTArray([-2.04204467, -2.00834275, -1.90356182, -1.90356182, -2.07693764,
          -1.86450489, -1.84551951, -1.88384714, -2.00834275, -1.78167794,
          -1.85496853, -1.74686788, -1.79952371, -1.65660066, -1.90356182,
          -1.70489805, -1.58054701, -1.65660066, -1.72988182, -1.65660066,
          -1.79952371, -1.68855875, -1.72148906, -1.55870579, -1.5732195 ,
          -1.53022366, -1.5732195 , -1.5443757 , -1.50243276, -1.56593934,
          -1.55870579, -1.43572601, -1.36030918, -1.48878393, -1.44876848,
          -1.49558829, -1.3124866 , -1.39743105, -1.38493132, -1.42282479,
          -1.27777357, -1.34818035, -1.22191364, -1.26076301, -1.30081018,
          -1.22191364, -1.24397231, -1.21645592, -1.21645592, -1.13199818,
          -1.09161697, -1.12688723, -1.16830898, -1.12179475, -1.05233487,
          -1.14227629, -1.03786953, -1.09161697, -0.90900972, -1.16306304,
          -0.98134919, -1.01406547, -1.00934905, -0.86104625, -0.96295715,
          -0.86104625, -0

In [1428]:
quadrature_func2(np.array([1.2, 1.3]), 0.3, dataset[:, 0], dr)

array([4.7658382e-11, 1.5550896e-11])

In [1284]:
integrate.fixed_quad(quadrature_func, a[1], 4, (ax[50], dataset[:, 0], dr), n=21)

(6.804477371811252e-63, None)

In [1859]:
def min_func(theta, ndx):
    return -integrate.fixed_quad(quadrature_func, a[1], 4, (theta, discr_guess, dataset2[:, ndx], dr), n=12)[0]

In [1822]:
from scipy import optimize
discr_guess

1.3725827336524024

In [1860]:
otpt = list(map(lambda x: optimize.fminbound(min_func, -2.5, 6, args=(x,)), range(dataset2.shape[1])))

In [1861]:
(abilities - otpt).mean()

ValueError: operands could not be broadcast together with shapes (1000,) (216,) 

In [1813]:
list(zip(abilities[:20], otpt))

[(-0.3846138209801119, -0.04610748563318766),
 (-1.842341014606915, -1.398161193327523),
 (0.2521595146596543, -0.1381240908114089),
 (3.7975195599401697, 4.999994710716215),
 (0.30534891879749393, 0.5429262695639332),
 (-0.0013869621560064045, 0.03850991698145206),
 (1.0452746572871523, 1.0550924368965546),
 (0.6318804816849865, 0.770621517505574),
 (-0.868599830555273, -0.48278291300671),
 (1.4822758149833606, 1.6064533983466973),
 (0.07508380494732467, 0.3257578616143445),
 (-0.54417428885446, -0.6721809006720944),
 (-0.4206959663812981, -0.9355587604394155),
 (0.114182757836806, 0.29999326612635024),
 (-1.525856223736279, -1.9012657452339883),
 (0.05175020926912631, 0.12330210787999082),
 (0.4823688305129623, 0.5309630869374579),
 (-0.464436454421467, 0.2202103401074848),
 (-1.8987461706163553, -1.6032131624774348),
 (0.4296579335725527, 0.22371227308870323)]

In [1840]:
hx, xx = np.histogram(otpt, 10)

In [1830]:
def min_disc(guess):
    return -np.log(discriminate_func(guess, otpt, dataset, dr)).sum()

optimize.fminbound(min_disc, 1, 3)

1.4925686889458196

In [1792]:
discrimination

array([0.87])

In [1841]:
plt = figure(plot_width=600, plot_height=400)
plt.line((xx[1:] + xx[:-1])*0.5, hx)
show(plt)

In [1483]:
dr * np.sqrt(1 + np.pi * 1.378**2 / 8) / 1.378

array([1.66315969, 0.96341006, 0.73155422, 0.83073453, 0.23901596,
       0.74929838, 0.34105357, 0.66172418, 0.73155422, 2.48015383])

In [1837]:
nf = np.array(otpt)

In [1848]:
np.unique(dataset, axis=1, return_counts=True)[1]

array([94,  7,  2,  2, 21,  1,  4,  5,  3,  2,  1,  1,  2,  1, 15,  2,  3,
        4, 10,  2,  3,  1,  3,  3,  1,  4,  2,  1,  9,  1,  1,  2,  3,  1,
        4,  1,  2,  1,  2,  1,  4,  1,  1,  3,  1,  1,  1,  1,  1,  3,  8,
        4,  4,  3,  3,  1,  3,  3,  1,  1,  1,  1,  3,  1,  2,  3,  1,  1,
        1,  3,  2,  2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  1,  1,  1,  1,
        1,  2,  3,  1,  1,  2,  1,  1,  1,  8,  4,  1,  4,  1,  3,  1,  6,
        1,  3,  3,  1,  2,  1,  2,  1,  2,  1,  1,  1,  1,  2,  1,  2,  1,
        1,  1,  1,  1,  2,  2,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  3,  1,  2,  1,  4,  2,
        1,  1,  1,  2,  1,  2,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  2,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  3,  1,  1,  2,  2,  1,  3])

In [1862]:
g = np.array([1.4183, 1.1445,1.1910, 1.6444,1.3398,1.4973,0.8748,1.5133,1.8716,2.1060])

In [1866]:
np.median(g)

1.4578

In [1275]:
.409 * np.sqrt(3.6/24)

0.15840501885988334

In [1277]:
syn_data

array([[ True, False, False, ...,  True,  True,  True],
       [False, False,  True, ...,  True,  True, False],
       [ True, False,  True, ...,  True, False,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [ True, False,  True, ..., False,  True, False]])

In [1278]:
from factoranalysis import contingency_table

In [1296]:
contingency_table(syn_data[0].astype('float') + 1, syn_data[1].astype('float')+1, start_val=1, stop_val=2)

array([[116.,  27.],
       [181., 176.]])

In [1281]:
list(zip(syn_data[0], syn_data[1]))

[(True, False),
 (False, False),
 (False, True),
 (True, False),
 (False, False),
 (False, False),
 (True, True),
 (True, True),
 (True, False),
 (True, False),
 (False, True),
 (False, False),
 (True, True),
 (True, True),
 (True, True),
 (False, True),
 (True, False),
 (False, False),
 (False, False),
 (False, True),
 (True, False),
 (True, True),
 (True, True),
 (True, False),
 (True, False),
 (True, True),
 (False, True),
 (False, False),
 (True, False),
 (True, True),
 (False, False),
 (True, True),
 (True, True),
 (True, False),
 (False, False),
 (True, True),
 (False, False),
 (True, False),
 (True, True),
 (True, False),
 (True, False),
 (True, False),
 (True, False),
 (True, True),
 (True, True),
 (False, False),
 (True, False),
 (True, False),
 (False, False),
 (True, True),
 (False, False),
 (False, True),
 (True, True),
 (True, False),
 (False, False),
 (True, False),
 (True, False),
 (True, True),
 (True, False),
 (True, False),
 (True, False),
 (True, False),
 (False, Fal

In [216]:
x = np.linspace(-6, 6, 10001)
scalar = 10001 / 12

p1 = irt_evaluation(np.array([2.3]), np.array([1.7]), x).squeeze()
p1_new = irt_evaluation(np.array([2.5]), np.array([1.7]), x).squeeze()
p2 = irt_evaluation(np.array([.3]), np.array([1.7]), x).squeeze()
g  = 1.0 / np.sqrt(2 * np.pi) * np.exp(-x * x / 2) / scalar

prdct = p1 * p2 * g

In [208]:
(p1 * p2 * g).sum() / (p2 * g).sum()

0.10586060711336329

In [209]:
np.argmin(np.abs(p1 - 0.10586060711336329))

5871

In [217]:
(p1_new * p2 * g).sum() / (p2 * g).sum()

0.08199109441169902

In [218]:
np.argmin(np.abs(p1_new - 0.08199109441169902))

5899

In [214]:
x[5871]

1.0451999999999995

In [219]:
x[5899]

1.0787999999999993